In [ ]:
import os,sys
sys.path.insert(0,'..')
import numpy as np
from rsreader.netreader.SegReader import SegReader
from rsreader.netreader.mxreader.GluonDataset import GluonDataset
from rsreader.netreader.SegReader import SegReader
from rsreader.utility.Normalization import *
from rsreader.utility.DataAugmentation import *
from rsreader.utility.ImageTrans import ImageTrans
from rsreader.utility.JointTrans import JointTrans
from rsreader.utility.LabelViz import LabelViz
import random
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def viz(img):
    img = np.swapaxes(img,0,2) #c,h,w -- > w,h,c
    img = np.swapaxes(img,0,1)
    return img

In [ ]:
num_classes = 6
patchsize = (2048, 2048)
step = (512,512)
dSize = (512,512)
checkperoid = 1
nepoch = 100
niter = 0
lr = 0.001
weight_decay = 0.00005
momentum = 0.9
printperoid = 100
samplerate = 8

In [ ]:
mean = [86.31373922, 92.0910872, 85.50070918, 95.87856063]
std = [36.3780667, 35.29996454, 36.60494421, 36.4936119]
# norm = standard_transforms.Normalize(mean=mean,std=std)

mirror = Mirror(patchsize=patchsize)
rotate = Rotate(patchsize=patchsize)
randomcrop = RandomCrop(scale_range=[0.25, 1.0], dSize=dSize,symmetry=True, eps = 1000)
scale = ScaleAug(patchsize=patchsize, symmetry=True, scale_range=[0.25, 2.0], eps=1000)
# joint_trans = JointTrans(funcs=[mirror, rotate, scale])
joint_trans = JointTrans(funcs=[scale,mirror, rotate], docopy = False)
joint_trans = JointTrans(funcs=[rotate],docopy=False)

norm = Normalization(mean=mean,std = std)
img_trans = ImageTrans(trans=[norm])
img_trans = ImageTrans(trans=[])
gt_trans = ImageTrans(trans=[])

In [ ]:
netreader = SegReader(flist_name='/home/chenkq/data/potsdam/trainset3.lst',
                      data_root='/home/chenkq/data/potsdam',
                      batchsize=1, cropsize=patchsize, step=step,
                      img_trans=img_trans, gt_trans=gt_trans, joint_trans=joint_trans,
                      withgt=True, bandlist=None, sampleseed=-1,
                      lvreadertype='gdal', parsertype='common', openfirstly=True)

In [ ]:
train_set = GluonDataset(netreader, niter=-1, mode='shuffle')

In [ ]:
length = len(train_set)
idx = random.sample(range(0,length),1)[0]
idx = 500
image,label = train_set[idx]
print(image.shape)
img = image
gt = label
img_viz = viz(img[0:3]).astype('uint8')
converter = LabelViz('../rsreader/utility/isprs.txt')
gt_viz = converter.Viz(gt)
print(img_viz.shape, gt_viz.shape)
plt.figure(figsize=(20,10))
plt.subplot(131)
plt.imshow(img_viz)
plt.subplot(132)
plt.imshow(gt_viz)
plt.subplot(133)
composition = 0.6*img_viz + 0.4*gt_viz
plt.imshow(composition.astype('uint8'))

## Calculate Mean and deviation

In [ ]:
joint_trans = JointTrans([])
img_trans = ImageTrans([])
gt_trans = ImageTrans(trans=[])
reader = SegReader(flist_name='/home/chenkq/data/vaihingen/trainset_irrgndsm.lst',
                      data_root='/home/chenkq/data/vaihingen/',
                      batchsize=1, cropsize=-1, step=-1,
                      img_trans=img_trans, gt_trans=gt_trans, joint_trans=joint_trans,
                      withgt=True, bandlist=None, sampleseed=-1,
                      lvreadertype='gdal', parsertype='common', openfirstly=True)

In [ ]:
means = []
insts = []
for idx in range(0, len(reader.readerStore.lines)):
    top,gt = reader[idx]
    img = top
#     print(img.shape)
    m = np.mean(img,axis=(1,2))
    means.append(m)
    inst = img.size/img.shape[0]
    insts.append(inst)
    print('#:', inst, ', mean:', m)

In [ ]:
means = np.asarray(means)
mean = np.average(means,axis=0,weights=insts)
print('mean=',mean)

In [ ]:
stds = []
for idx in range(0, len(reader.readerStore.lines)):
    top,gt = reader[idx]
    img = top
    img = np.swapaxes(img,0,2)
    img = np.swapaxes(img,0,1)
    sq = (img-mean)*(img-mean)
    stds.append(np.sum(sq,axis=(0,1)))
    print('#: ', insts[idx], stds[idx])

In [ ]:
stds = np.asarray(stds)
std = np.sum(stds, axis=0)
print(std)
std /= np.sum(insts)
print(np.sqrt(std))